#Decision Tree

In [2]:
# 1. Importação de bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# 2. Carregamento dos dados
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')



/var/folders/j_/wwm3syvn58nb_fbllrh1fxg40000gn/T/ipykernel_7639/2495195231.py:12: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


In [3]:
# 3. Defina o nome da coluna alvo
TARGET = 'Credit_Score'  # ajuste se necessário

# 4. Separe X e y
X = train.drop(columns=[TARGET])
y = train[TARGET]


In [4]:

# 5. Divisão treino / validação
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [5]:

# 6. Identifique features numéricas e categóricas
numeric_feats = X_train.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_feats = X_train.select_dtypes(include=['object','category']).columns.tolist()


In [6]:
from sklearn.preprocessing import FunctionTransformer

# 7. Pipelines de pré‑processamento
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler',  StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    # converte tudo para string, unificando o tipo
    ('to_str',   FunctionTransformer(lambda X: X.astype(str))),
    ('onehot',   OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_feats),
    ('cat', categorical_pipeline, categorical_feats)
])




In [7]:
# 8. Pipeline completo com Decision Tree
dt_pipe = Pipeline([
    ('preproc', preprocessor),
    ('clf',     DecisionTreeClassifier(random_state=42))
])


In [1]:
# 9. Treinamento
dt_pipe.fit(X_train, y_train)

NameError: name 'dt_pipe' is not defined

In [ ]:
# 10. Avaliação no conjunto de validação
y_pred = dt_pipe.predict(X_val)
# Se quiser probabilidade, usar dt_pipe.predict_proba(X_val)[:,1]
print("=== Decision Tree ===")
print(classification_report(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))
if hasattr(dt_pipe, "predict_proba"):
    print("ROC AUC: ", roc_auc_score(y_val, dt_pipe.predict_proba(X_val)[:,1]))

# 11. Treinar no conjunto completo e gerar submissão
dt_pipe.fit(X, y)  # re-treina usando todo o train.csv

test_preds = dt_pipe.predict(test)
submission = pd.DataFrame({
    'ID': test.index,      # ou substitua pela coluna de ID em test.csv
    TARGET: test_preds
})
submission.to_csv('dt_predictions.csv', index=False)
print("\nArquivo dt_predictions.csv gerado com sucesso!")